In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

## Importing Data

In [2]:
# Add column with service name to each DataFrame
amazon = pd.read_csv("data/amazon_clean.csv").assign(service="amazon")
disney = pd.read_csv("data/disney_clean.csv").assign(service="disney")
hulu = pd.read_csv("data/hulu_clean.csv").assign(service="hulu")
netflix = pd.read_csv("data/netflix_clean.csv").assign(service="netflix")

In [3]:
raw_dfs = [amazon,disney,hulu,netflix]

## Initial Restriction and Combining

In [4]:
# Combine all DataFrames
services = raw_dfs[0].copy()
for df in raw_dfs[1:]:
    services = sqldf("SELECT * FROM services UNION SELECT * FROM df")

In [5]:
services.head()

,type,title,country,date_added,release_year,rating,duration,listed_in,description,service
0,Movie,"""Mixed Up""",None,None,2020,16+,106,"Drama,Romance","""Mixed Up"" examines casual factors that make u...",amazon
1,Movie,#Alive,South Korea,"September 8, 2020",2020,TV-MA,99,"Horror,International,Thriller","As a grisly virus rampages a city, a lone man ...",netflix
2,Movie,#AnneFrank - Parallel Stories,Italy,"July 1, 2020",2019,TV-14,95,"Documentary,International","Through her diary, Anne Frank's story is retol...",netflix
3,Movie,#FriendButMarried,Indonesia,"May 21, 2020",2018,TV-G,102,"Drama,International,Romance","Pining for his high school crush for years, a ...",netflix
4,Movie,#FriendButMarried 2,Indonesia,"June 28, 2020",2020,TV-G,104,"Drama,International,Romance",As Ayu and Ditto finally transition from best ...,netflix


In [6]:
# Separate DataFrames based on content type 
movies = sqldf("SELECT * FROM services WHERE type == \"Movie\"")
tv = sqldf("SELECT * FROM services WHERE type == \"TV Show\"")

## Basic Stats Computation

In [7]:
# Find the mean release year of content as well as the total number of movies on each service
basic_movie_stats = sqldf("SELECT AVG(release_year) as mean_year, COUNT(*) as n_titles, service FROM movies GROUP BY service")
basic_movie_stats

,mean_year,n_titles,service
0,2006.875577,7804,amazon
1,1999.169685,1049,disney
2,2011.942174,1297,hulu
3,2013.108824,6074,netflix


In [8]:
# Find the mean release year of content as well as the total number of movies on each service
basic_tv_stats = sqldf("SELECT AVG(release_year) as mean_year, COUNT(*) as n_titles, service FROM tv GROUP BY service")
basic_tv_stats

,mean_year,n_titles,service
0,2014.385921,1591,amazon
1,2013.253165,395,disney
2,2012.530522,1589,hulu
3,2016.709398,2660,netflix


## Genre Breakdown

In [9]:
# Get a list of all genres that appear in movies
movie_genres = []
for index, row in movies.iterrows():
    try:
        for genre in row["listed_in"].split(","):
            if(genre not in movie_genres):
                movie_genres.append(genre)
    except:
        pass
        
            
tv_genres = []
for index, row in tv.iterrows():
    try:
        for genre in row["listed_in"].split(","):
            if(genre not in tv_genres):
                tv_genres.append(genre)
    except:
        pass

In [ ]:
# Append each instance of a genre to the service_genres DataFrame
service_genres = pd.DataFrame(columns = ["service", "type", "genre"])
for index, row in services.iterrows():
    try:
        for genre in row["listed_in"].split(","):
            service_genres = service_genres.append({"service": row["service"], "type": row["type"], "genre": genre}, ignore_index=True)
    except:
        pass

In [ ]:
service_genres.shape

In [ ]:
# Count the instances of a genre for each service and type
service_genres_counted = sqldf("SELECT service, type, genre, COUNT(*) as count FROM service_genres GROUP BY service, type, genre")

In [ ]:
# Separate movie genres from TV genres
service_genres_movies = sqldf("SELECT * FROM service_genres_counted WHERE type == \"Movie\"")

In [ ]:
# Separate TV genres from movie genres
service_genres_tv = sqldf("SELECT * FROM service_genres_counted WHERE type == \"TV Show\"")

In [ ]:
# Find genres that only appear on one service
single_service_movie_genres = sqldf("SELECT * FROM (SELECT genre, COUNT(DISTINCT(service)) as num_services FROM service_genres_movies GROUP BY genre) WHERE num_services == 1")
single_service_tv_genres = sqldf("SELECT * FROM (SELECT genre, COUNT(DISTINCT(service)) as num_services FROM service_genres_tv GROUP BY genre) WHERE num_services == 1")

In [ ]:
# Sort each service's genres by count
service_genres_movies = sqldf("SELECT * FROM service_genres_movies ORDER BY service, count DESC")
service_genres_movies

In [ ]:
# Get the top 5 genres on each service
limit = 5
frames = []
for service in np.unique(service_genres_movies["service"]):
    frames.append(sqldf(f"SELECT * FROM (SELECT * FROM service_genres_movies ORDER BY service, count DESC) WHERE service == \"{service}\" LIMIT {limit}"))
    
top_by_genre = frames[0].copy()
for frame in frames[1:]:
    top_by_genre = sqldf("SELECT * FROM top_by_genre UNION SELECT * FROM frame")
top_by_genre = sqldf("SELECT * FROM top_by_genre ORDER BY service, count DESC")
top_by_genre

In [ ]:
# Get top service for each genre
top_each_genre = sqldf("SELECT genre, service as top_service, MAX(count) as count FROM service_genres_movies WHERE genre NOT IN (SELECT genre FROM single_service_movie_genres) GROUP BY genre")
top_each_genre

## Ratings

G, PG, PG-13, R, etc. 

In [ ]:
ratings_by_service = sqldf("SELECT service, type, COUNT(rating) as count, rating FROM services GROUP BY service, rating")
ratings_by_service

In [ ]:
movie_ratings_by_service = sqldf("SELECT * FROM ratings_by_service WHERE type == \"Movie\"")
movie_ratings_by_service